Creazione BASE DATI dataset finale

Dependencies

In [1]:
import pandas as pd
import numpy as np 

Datasets

In [42]:
p05 = pd.read_csv("./cig_pnrr_anac.csv/cig_pnrr_anac.csv",sep=",")
p03 = pd.read_csv("./datasets/misurepremiali-pnrrpnc_csv.csv",delimiter=';')
p04 = pd.read_csv("./datasets/indicatori-pnrrpnc_csv.csv",delimiter=';')

C:\Users\Alessandra\AppData\Local\Temp\ipykernel_12308\1520190168.py:1: DtypeWarning: Columns (51) have mixed types. Specify dtype option on import or set low_memory=False.
  p05 = pd.read_csv("./cig_pnrr_anac.csv/cig_pnrr_anac.csv",sep=",")


In [43]:
p01 = pd.read_csv("./datasets/cup_csv.csv",delimiter=';',encoding = "ISO-8859-1")
p02 = pd.read_excel("./datasets/CUP_PNRR_2023_12_08.xlsx")
comuni = pd.read_csv("./datasets/Elenco-comuni-italiani.csv",delimiter=';',encoding = "ISO-8859-1")
missioni_map = pd.read_json("./missioni_pnrr.json", orient="index")
componenti_map = pd.read_json("./componenti.json", orient="index")

Pre-processing dei dataset

In [44]:
# preprocessing
p05 = p05.rename(columns={'cig':'CIG'})
p03 = p03.rename(columns={'cig':'CIG'})
p04 = p04.rename(columns={'cig':'CIG'})

In [45]:
# pre-processing p03
# add colonna 'CATEGORIA'
def conditions(s):
    if s['cod_mis_premiale'] in [1,2,9]:
        return "GENERALE"
    elif s['cod_mis_premiale'] in [3,8,12]:
        return "DISABILI"
    elif s['cod_mis_premiale'] in [4,6,10]:
        return "GENERE"
    elif s['cod_mis_premiale'] in [5,7,11]:
        return "ETÀ"
    else:
        return 0
    
p03['CATEGORIA'] = p03.apply(conditions,axis=1)

In [46]:
# pre-processing p05 pnew
# add colonna 'CLASSE_IMPORTO' 
pd.set_option('display.float_format', lambda x: '%.2f' % x)
def map_importo(importo):
    if importo <= 100000:
        return 'BASSA'
    elif importo <= 1000000:
        return 'MEDIA'
    else:
        return 'ALTA'
    
p05['CLASSE_IMPORTO'] = p05['importo_complessivo_gara'].apply(map_importo)
# add colonna 'REGIONI'
prov_reg = comuni[['Denominazione Regione','Denominazione dell\'Unità territoriale sovracomunale \n(valida a fini statistici)']]
prov_reg = prov_reg.rename(columns={'Denominazione dell\'Unità territoriale sovracomunale \n(valida a fini statistici)':'Denominazione sovracomunale','Denominazione Regione':'Regione'})
prov_reg['Denominazione sovracomunale']=prov_reg['Denominazione sovracomunale'].replace(to_replace=r"Valle d'Aosta/Vallée d'Aoste", value='Aosta', regex=False)
prov_reg['Denominazione sovracomunale']=prov_reg['Denominazione sovracomunale'].replace(to_replace=r"Bolzano/Bozen", value='Bolzano',regex=False)
prov_reg['Denominazione sovracomunale']=prov_reg['Denominazione sovracomunale'].str.upper()
prov_reg=prov_reg.drop_duplicates()
p05=p05.merge(prov_reg,left_on='provincia', right_on='Denominazione sovracomunale',how='left')
p05=p05.drop_duplicates()
#p05_new = p05[['CIG','importo_complessivo_gara','CLASSE_IMPORTO', 'luogo_istat', 'provincia', 'Denominazione sovracomunale','Regione','data_pubblicazione']]

In [47]:
p05.columns

Index(['CIG', 'cig_accordo_quadro', 'numero_gara', 'oggetto_gara',
       'importo_complessivo_gara', 'n_lotti_componenti', 'oggetto_lotto',
       'importo_lotto', 'oggetto_principale_contratto', 'stato', 'settore',
       'luogo_istat', 'provincia', 'data_pubblicazione',
       'data_scadenza_offerta', 'cod_tipo_scelta_contraente',
       'tipo_scelta_contraente', 'cod_modalita_realizzazione',
       'modalita_realizzazione', 'codice_ausa',
       'cf_amministrazione_appaltante',
       'denominazione_amministrazione_appaltante', 'sezione_regionale',
       'id_centro_costo', 'denominazione_centro_costo', 'anno_pubblicazione',
       'mese_pubblicazione', 'cod_cpv', 'descrizione_cpv', 'flag_prevalente',
       'COD_MOTIVO_CANCELLAZIONE', 'MOTIVO_CANCELLAZIONE',
       'DATA_CANCELLAZIONE', 'DATA_ULTIMO_PERFEZIONAMENTO',
       'COD_MODALITA_INDIZIONE_SPECIALI', 'MODALITA_INDIZIONE_SPECIALI',
       'COD_MODALITA_INDIZIONE_SERVIZI', 'MODALITA_INDIZIONE_SERVIZI',
       'DURATA_PREVIST

In [7]:
p05['importo_complessivo_gara'].dtype
p02['COSTO_PROGETTO'].dtype
p02['IMPORTO_FINANZIAMENTO'].dtype

dtype('float64')

Servono codice componente, componente, investimento?

In [48]:
# pre-processing p02
p02["CODICE_COMPONENTE"] = p02["NOME_TEMATICA"].apply(lambda x: x[:4])
p02["CODICE_MISSIONE"] = p02["NOME_TEMATICA"].apply(lambda x: x[:2])
# add colonna 'MISSIONE', 'COMPONENTE'


In [49]:
missioni_map.columns = ["MISSIONE"]
componenti_map.columns = ["COMPONENTE"]

In [50]:
p02 = p02.merge(missioni_map,left_on="CODICE_MISSIONE", right_on=missioni_map.index, how='inner')
p02 = p02.drop_duplicates()
p02 = p02.merge(componenti_map,left_on="CODICE_COMPONENTE", right_on=componenti_map.index, how='inner')
p02 = p02.drop_duplicates()

In [51]:
p02

,CODICE_CUP,CODICE_TEMATICA,NOME_TEMATICA,CODICE_TEMPLATE,DESCRIZIONE_TEMPLATE,CODICE_NATURA,DESCRIZIONE_NATURA,CODICE_TIPOLOGIA,DESCRIZIONE_TIPOLOGIA,DESCRIZIONE_AGGREGATA,...,ANNO_DECISIONE,DATA_GENERAZIONE_COMPLETO,COSTO_PROGETTO,IMPORTO_FINANZIAMENTO,STATO_PROGETTO,TIPO_PROGETTO,CODICE_COMPONENTE,CODICE_MISSIONE,MISSIONE,COMPONENTE
0,D97H23002600004,432,M2C1: Agricoltura sostenibile ed Economia Circ...,NaN,NaN,7,CONCESSIONE DI INCENTIVI AD UNITA' PRODUTTIVE,5,NUOVA REALIZZAZIONE CON EFFICIENTAMENTO ENERGE...,VILLARI VITO FRANCESCO*Realizzazione di un imp...,...,2023,2023-12-08,67840.00,54272.00,Attivo,Progetto Normale,M2C1,M2,RIVOLUZIONE VERDE E TRANSIZIONE ECOLOGICA,Agricoltura sostenibile ed economia circolare
1,D87H23003000004,432,M2C1: Agricoltura sostenibile ed Economia Circ...,NaN,NaN,7,CONCESSIONE DI INCENTIVI AD UNITA' PRODUTTIVE,5,NUOVA REALIZZAZIONE CON EFFICIENTAMENTO ENERGE...,DE FORTUNA ROBERTO*Realizzazione di un impiant...,...,2023,2023-12-08,25000.00,20000.00,Attivo,Progetto Normale,M2C1,M2,RIVOLUZIONE VERDE E TRANSIZIONE ECOLOGICA,Agricoltura sostenibile ed economia circolare
2,D87H23002990004,432,M2C1: Agricoltura sostenibile ed Economia Circ...,NaN,NaN,7,CONCESSIONE DI INCENTIVI AD UNITA' PRODUTTIVE,5,NUOVA REALIZZAZIONE CON EFFICIENTAMENTO ENERGE...,LEGGIO SALVATORE*Realizzazione di un impianto ...,...,2023,2023-12-08,20500.00,16400.00,Attivo,Progetto Normale,M2C1,M2,RIVOLUZIONE VERDE E TRANSIZIONE ECOLOGICA,Agricoltura sostenibile ed economia circolare
3,D37H23002980004,432,M2C1: Agricoltura sostenibile ed Economia Circ...,NaN,NaN,7,CONCESSIONE DI INCENTIVI AD UNITA' PRODUTTIVE,5,NUOVA REALIZZAZIONE CON EFFICIENTAMENTO ENERGE...,RAINER FRANZ JOSEF*Realizzazione di un impiant...,...,2023,2023-12-08,71715.00,57372.00,Attivo,Progetto Normale,M2C1,M2,RIVOLUZIONE VERDE E TRANSIZIONE ECOLOGICA,Agricoltura sostenibile ed economia circolare
4,D87H23002980004,432,M2C1: Agricoltura sostenibile ed Economia Circ...,NaN,NaN,7,CONCESSIONE DI INCENTIVI AD UNITA' PRODUTTIVE,5,NUOVA REALIZZAZIONE CON EFFICIENTAMENTO ENERGE...,AZIENDA AGRICOLA DRUGOLO S.R.L.*Realizzazione ...,...,2023,2023-12-08,1680729.00,1344583.00,Attivo,Progetto Normale,M2C1,M2,RIVOLUZIONE VERDE E TRANSIZIONE ECOLOGICA,Agricoltura sostenibile ed economia circolare
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358936,J71H92000000007,476,M3C1: Investimenti sulla rete ferroviaria - I1...,NaN,NaN,3,REALIZZAZIONE DI LAVORI PUBBLICI (OPERE ED IMP...,58,AMPLIAMENTO (COMPRESA SOPRAELEVAZIONE) O POTEN...,LINEA ANCONA-FOGGIA-BARI*TERRITORIO MULTIREGIO...,...,1992,2003-10-31,458000000.00,458000000.00,Attivo,Progetto Normale,M3C1,M3,INFRASTRUTTURE PER UNA MOBILITÀ SOSTENIBILE,Rete ferroviaria ad alta velocità/capacità e s...
358937,J71H92000000007,476,M3C1: Investimenti sulla rete ferroviaria - I1...,NaN,NaN,3,REALIZZAZIONE DI LAVORI PUBBLICI (OPERE ED IMP...,58,AMPLIAMENTO (COMPRESA SOPRAELEVAZIONE) O POTEN...,LINEA ANCONA-FOGGIA-BARI*TERRITORIO MULTIREGIO...,...,1992,2003-10-31,458000000.00,458000000.00,Attivo,Progetto Normale,M3C1,M3,INFRASTRUTTURE PER UNA MOBILITÀ SOSTENIBILE,Rete ferroviaria ad alta velocità/capacità e s...
358938,J71H92000000007,476,M3C1: Investimenti sulla rete ferroviaria - I1...,NaN,NaN,3,REALIZZAZIONE DI LAVORI PUBBLICI (OPERE ED IMP...,58,AMPLIAMENTO (COMPRESA SOPRAELEVAZIONE) O POTEN...,LINEA ANCONA-FOGGIA-BARI*TERRITORIO MULTIREGIO...,...,1992,2003-10-31,458000000.00,458000000.00,Attivo,Progetto Normale,M3C1,M3,INFRASTRUTTURE PER UNA MOBILITÀ SOSTENIBILE,Rete ferroviaria ad alta velocità/capacità e s...
358939,J71H92000000007,476,M3C1: Investimenti sulla rete ferroviaria - I1...,NaN,NaN,3,REALIZZAZIONE DI LAVORI PUBBLICI (OPERE ED IMP...,58,AMPLIAMENTO (COMPRESA SOPRAELEVAZIONE) O POTEN...,LINEA ANCONA-FOGGIA-BARI*TERRITORIO MULTIREGIO...,...,1992,2003-10-31,458000000.00,458000000.00,Attivo,Progetto Normale,M3C1,M3,INFRASTRUTTURE PER UNA MOBILITÀ SOSTENIBILE,Rete ferroviaria ad alta velocit

In [28]:
# pre-processing p02
# add colonna 'CODICE COMPONENTE', 'CODICE MISSIONE', 'COMPONENTE', 'INVESTIMENTO' 
def map_comp(tematica):
    if tematica.count('-')==1:
        componente = tematica[6:tematica.find('-')-1]
    else:
        componente = tematica[6:tematica.find('I')-4]
    return componente

def map_investimento(tematica):
    # con numeri
    if tematica.count('-')==1:
        investimento = tematica[tematica.find('-')+2:]
    else:
        first = tematica.find('-')+1
        tematica = tematica[first:]
        investimento = tematica[tematica.find('-')+2:]
    return investimento

p02["CODICE_COMPONENTE"] = p02["NOME_TEMATICA"].apply(lambda x: x[:4])
p02["CODICE_MISSIONE"] = p02["NOME_TEMATICA"].apply(lambda x: x[:2])
p02["COMPONENTE"] = p02['NOME_TEMATICA'].apply(map_comp)
p02["INVESTIMENTO"] = p02['NOME_TEMATICA'].apply(map_investimento)
# add colonna 'MISSIONE'
missioni_map.columns = ["MISSIONE"]
p02 = p02.merge(missioni_map,left_on="CODICE_MISSIONE", right_on=missioni_map.index, how='inner')
p02 = p02.drop_duplicates()

In [32]:
p02.columns

Index(['CODICE_CUP', 'CODICE_TEMATICA', 'NOME_TEMATICA', 'CODICE_TEMPLATE',
       'DESCRIZIONE_TEMPLATE', 'CODICE_NATURA', 'DESCRIZIONE_NATURA',
       'CODICE_TIPOLOGIA', 'DESCRIZIONE_TIPOLOGIA', 'DESCRIZIONE_AGGREGATA',
       'SOGGETTO_TITOLARE', 'PIVA_SOGGETTO', 'UNITA_ORGANIZZATIVA',
       'STRUMENTO_PROGRAMMAZIONE', 'CODICE_STATO', 'STATO', 'CODICE_REGIONE',
       'REGIONE', 'CODICE_PROVINCIA', 'PROVINCIA', 'CODICE_COMUNE', 'COMUNE',
       'CODICE_SETTORE_INTERVENTO', 'SETTORE_INTERVENTO',
       'CODICE_SOTTOSETTORE_INTERVENTO', 'SOTTOSETTORE_INTERVENTO',
       'CODICE_CATEGORIA_INTERVENTO', 'CATEGORIA_INTERVENTO', 'ANNO_DECISIONE',
       'DATA_GENERAZIONE_COMPLETO', 'COSTO_PROGETTO', 'IMPORTO_FINANZIAMENTO',
       'STATO_PROGETTO', 'TIPO_PROGETTO', 'CODICE_COMPONENTE',
       'CODICE_MISSIONE', 'MISSIONE', 'COMPONENTE'],
      dtype='object')

In [85]:
#p05[p05['CIG']=="A0011137E1"]

,CIG,cig_accordo_quadro,numero_gara,oggetto_gara,importo_complessivo_gara,n_lotti_componenti,oggetto_lotto,importo_lotto,oggetto_principale_contratto,stato,...,IPOTESI_COLLEGAMENTO,CIG_COLLEGAMENTO,COD_ESITO,ESITO,DATA_COMUNICAZIONE_ESITO,FLAG_PNRR_PNC,filename,CLASSE_IMPORTO,Regione,Denominazione sovracomunale
0,A0011137E1,NaN,9264014,FORNITURA DI CURVE IN ACCIAIO A SALDARE DN. 80...,21131.90,1,FORNITURA DI CURVE IN ACCIAIO A SALDARE DN. 80...,21131.90,FORNITURE,ATTIVO,...,NaN,NaN,1.00,AGGIUDICATA,2023-08-29,1,cig_csv_2023_08.csv.gz,BASSA,NaN,NaN


joins

In [52]:
# left join p5 e p3
leftnew_3 = p05.merge(p03, on='CIG',how='left')
leftnew_3 = leftnew_3.drop_duplicates()
print(leftnew_3.columns)
print(leftnew_3.shape)

Index(['CIG', 'cig_accordo_quadro', 'numero_gara', 'oggetto_gara',
       'importo_complessivo_gara', 'n_lotti_componenti', 'oggetto_lotto',
       'importo_lotto', 'oggetto_principale_contratto', 'stato', 'settore',
       'luogo_istat', 'provincia', 'data_pubblicazione',
       'data_scadenza_offerta', 'cod_tipo_scelta_contraente',
       'tipo_scelta_contraente', 'cod_modalita_realizzazione',
       'modalita_realizzazione', 'codice_ausa',
       'cf_amministrazione_appaltante',
       'denominazione_amministrazione_appaltante', 'sezione_regionale',
       'id_centro_costo', 'denominazione_centro_costo', 'anno_pubblicazione',
       'mese_pubblicazione', 'cod_cpv', 'descrizione_cpv', 'flag_prevalente',
       'COD_MOTIVO_CANCELLAZIONE', 'MOTIVO_CANCELLAZIONE',
       'DATA_CANCELLAZIONE', 'DATA_ULTIMO_PERFEZIONAMENTO',
       'COD_MODALITA_INDIZIONE_SPECIALI', 'MODALITA_INDIZIONE_SPECIALI',
       'COD_MODALITA_INDIZIONE_SERVIZI', 'MODALITA_INDIZIONE_SERVIZI',
       'DURATA_PREVIST

In [59]:
#leftnew_3[leftnew_3['CIG']=="A0011137E1"]

,CIG,importo_complessivo_gara,CLASSE_IMPORTO,luogo_istat,provincia,Denominazione sovracomunale,Regione,data_pubblicazione,cod_mis_premiale,misura_premiale,CATEGORIA
0,A0011137E1,21131.90,BASSA,NaN,NaN,NaN,NaN,2023-08-02,NaN,NaN,NaN


In [53]:
# left join il risultato con p4
leftnew_34 = leftnew_3.merge(p04, on='CIG', how='left')
leftnew_34 = leftnew_34.drop_duplicates()
print(leftnew_34.columns)
print(leftnew_34.shape)

Index(['CIG', 'cig_accordo_quadro', 'numero_gara', 'oggetto_gara',
       'importo_complessivo_gara', 'n_lotti_componenti', 'oggetto_lotto',
       'importo_lotto', 'oggetto_principale_contratto', 'stato', 'settore',
       'luogo_istat', 'provincia', 'data_pubblicazione',
       'data_scadenza_offerta', 'cod_tipo_scelta_contraente',
       'tipo_scelta_contraente', 'cod_modalita_realizzazione',
       'modalita_realizzazione', 'codice_ausa',
       'cf_amministrazione_appaltante',
       'denominazione_amministrazione_appaltante', 'sezione_regionale',
       'id_centro_costo', 'denominazione_centro_costo', 'anno_pubblicazione',
       'mese_pubblicazione', 'cod_cpv', 'descrizione_cpv', 'flag_prevalente',
       'COD_MOTIVO_CANCELLAZIONE', 'MOTIVO_CANCELLAZIONE',
       'DATA_CANCELLAZIONE', 'DATA_ULTIMO_PERFEZIONAMENTO',
       'COD_MODALITA_INDIZIONE_SPECIALI', 'MODALITA_INDIZIONE_SPECIALI',
       'COD_MODALITA_INDIZIONE_SERVIZI', 'MODALITA_INDIZIONE_SERVIZI',
       'DURATA_PREVIST

In [68]:
#leftnew_34[leftnew_34['CIG']=="A0011137E1"]

,CIG,importo_complessivo_gara,CLASSE_IMPORTO,luogo_istat,provincia,Denominazione sovracomunale,Regione,data_pubblicazione,cod_mis_premiale,misura_premiale,CATEGORIA,flag_quote,quota_femminile,quota_giovanile,cod_mot_deroga,mot_deroga
0,A0011137E1,21131.90,BASSA,NaN,NaN,NaN,NaN,2023-08-02,NaN,NaN,NaN,N,NaN,NaN,1.00,IMPORTO RIDOTTO DEL CONTRATTO


In [54]:
# left join il risultato con p1
leftnew_341 = leftnew_34.merge(p01, on='CIG', how='left')
leftnew_341 = leftnew_341.drop_duplicates()
print(leftnew_341.columns)
print(leftnew_341.shape)

Index(['CIG', 'cig_accordo_quadro', 'numero_gara', 'oggetto_gara',
       'importo_complessivo_gara', 'n_lotti_componenti', 'oggetto_lotto',
       'importo_lotto', 'oggetto_principale_contratto', 'stato', 'settore',
       'luogo_istat', 'provincia', 'data_pubblicazione',
       'data_scadenza_offerta', 'cod_tipo_scelta_contraente',
       'tipo_scelta_contraente', 'cod_modalita_realizzazione',
       'modalita_realizzazione', 'codice_ausa',
       'cf_amministrazione_appaltante',
       'denominazione_amministrazione_appaltante', 'sezione_regionale',
       'id_centro_costo', 'denominazione_centro_costo', 'anno_pubblicazione',
       'mese_pubblicazione', 'cod_cpv', 'descrizione_cpv', 'flag_prevalente',
       'COD_MOTIVO_CANCELLAZIONE', 'MOTIVO_CANCELLAZIONE',
       'DATA_CANCELLAZIONE', 'DATA_ULTIMO_PERFEZIONAMENTO',
       'COD_MODALITA_INDIZIONE_SPECIALI', 'MODALITA_INDIZIONE_SPECIALI',
       'COD_MODALITA_INDIZIONE_SERVIZI', 'MODALITA_INDIZIONE_SERVIZI',
       'DURATA_PREVIST

In [70]:
#leftnew_341[leftnew_341['CIG']=="A0011137E1"]

,CIG,importo_complessivo_gara,CLASSE_IMPORTO,luogo_istat,provincia,Denominazione sovracomunale,Regione,data_pubblicazione,cod_mis_premiale,misura_premiale,CATEGORIA,flag_quote,quota_femminile,quota_giovanile,cod_mot_deroga,mot_deroga,CUP
0,A0011137E1,21131.90,BASSA,NaN,NaN,NaN,NaN,2023-08-02,NaN,NaN,NaN,N,NaN,NaN,1.00,IMPORTO RIDOTTO DEL CONTRATTO,G79J19001530007


In [55]:
# left join il risultato con p2
leftnew_3412 = leftnew_341.merge(p02,how='left',left_on='CUP',right_on='CODICE_CUP')
leftnew_3412 = leftnew_3412.drop_duplicates()
print(leftnew_3412.columns)
print(leftnew_3412.shape)

Index(['CIG', 'cig_accordo_quadro', 'numero_gara', 'oggetto_gara',
       'importo_complessivo_gara', 'n_lotti_componenti', 'oggetto_lotto',
       'importo_lotto', 'oggetto_principale_contratto', 'stato',
       ...
       'ANNO_DECISIONE', 'DATA_GENERAZIONE_COMPLETO', 'COSTO_PROGETTO',
       'IMPORTO_FINANZIAMENTO', 'STATO_PROGETTO', 'TIPO_PROGETTO',
       'CODICE_COMPONENTE', 'CODICE_MISSIONE', 'MISSIONE', 'COMPONENTE'],
      dtype='object', length=112)
(273879, 112)


In [98]:
# sintesi in un solo block
# con P05 con tutte le colonne
leftnew_3_complete = p05.merge(p03, on='CIG',how='left')
leftnew_3_complete = leftnew_3_complete.drop_duplicates()
leftnew_34_complete = leftnew_3_complete.merge(p04, on='CIG', how='left')
leftnew_34_complete = leftnew_34_complete.drop_duplicates()
leftnew_341_complete = leftnew_34_complete.merge(p01, on='CIG', how='left')
leftnew_341_complete = leftnew_341_complete.drop_duplicates()
leftnew_3412_complete = leftnew_341_complete.merge(p02,how='left',left_on='CUP',right_on='CODICE_CUP')
leftnew_3412_complete = leftnew_3412_complete.drop_duplicates()

In [57]:
selezione = ['CIG', 'CUP', 'REGIONE', 'Regione', 'provincia', 'PROVINCIA', 'COMUNE',   'importo_complessivo_gara', 'CLASSE_IMPORTO', 'NOME_TEMATICA', 'CODICE_MISSIONE', 'MISSIONE', 'CODICE_COMPONENTE', 'COMPONENTE', 'cod_mis_premiale', 'misura_premiale', 'CATEGORIA',  'flag_quote', 'quota_femminile', 'quota_giovanile',
'cod_mot_deroga', 'mot_deroga', 'data_pubblicazione', 'anno_pubblicazione', 'FLAG_URGENZA', 'MOTIVO_URGENZA', 'ESITO']

In [58]:
p_basedati = leftnew_3412[selezione]

In [59]:
p_basedati[(p_basedati['REGIONE']!=p_basedati['Regione'].str.upper())&(p_basedati['Regione'].notna())&(p_basedati['REGIONE'].notna())]['REGIONE'].value_counts()

REGIONE
TRENTINO-ALTO ADIGE      4022
VALLE D'AOSTA             955
FRIULI-VENEZIA GIULIA     786
CAMPANIA                  616
LAZIO                     597
LOMBARDIA                 499
VENETO                    406
EMILIA-ROMAGNA            287
PUGLIA                    254
SICILIA                   247
PIEMONTE                  192
TOSCANA                   178
LIGURIA                    94
BASILICATA                 63
CALABRIA                   51
ABRUZZO                    27
SARDEGNA                   21
MOLISE                     18
MARCHE                     10
UMBRIA                      9
Name: count, dtype: int64

In [60]:
p_basedati.to_csv("basedati_new.csv", index = False, sep=';')

In [62]:
p_basedati['CIG'].nunique()

170481

In [66]:
p_basedati['MISSIONE'].nunique()

6

In [ ]:
p_basedati['misura_premiale']

In [3]:
p_basedati = pd.read_csv("basedati.csv",delimiter=';')

Quadrature

In [3]:
p_basedati[p_basedati["CATEGORIA"]=='GENERE']['CIG'].nunique()
p_basedati.nunique()
p_basedati['CIG'].nunique()

170481

In [4]:
p_basedati[p_basedati["CATEGORIA"]=='GENERE']["flag_quote"].value_counts()
p_basedati[p_basedati["CATEGORIA"]=='GENERE'].groupby("flag_quote")["CIG"].nunique()

flag_quote
N     787
Q    1586
S    3221
Name: CIG, dtype: int64

In [5]:
p_basedati[p_basedati["CATEGORIA"]=='GENERE']['CIG'].nunique()/p_basedati["CIG"].nunique()
#p_basedati[p_basedati["CATEGORIA"].notna()]['CIG'].nunique()/p_basedati["CIG"].nunique()

0.032818906505710316

check encoding quote_femminili

In [6]:
#pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.float_format', '{:.2f}'.format)



In [7]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
quote_fem = p_basedati[p_basedati['quota_femminile'].notna()]['quota_femminile'].unique()
quote_fem.min()
quote_fem.max()
quote_fem.mean()

13.424418604651162

In [11]:
p_basedati["quota_femminile_disc"]=pd.cut(x=p_basedati['quota_femminile'], bins=[0.0,5.0,10.0,15.0,20.0,25.0,30.0, float('inf')], labels=["0.-5%","5.-10%","10.-15%","15.-20%","20.-25%","25.-30%","err"])


In [4]:
p_basedati["quota_femminile_disc"].value_counts()

quota_femminile_disc
10.-15%    8253
0.-5%      6225
5.-10%     1626
15.-20%     275
25.-30%     138
20.-25%     109
err           1
Name: count, dtype: int64

In [5]:
print(p_basedati[p_basedati["quota_femminile"].notna()].nunique())
p_basedati[p_basedati["quota_femminile_disc"].notna()]

CIG                         4673
CUP                         3861
REGIONE                       20
Regione                       20
provincia                    106
PROVINCIA                    107
COMUNE                      1944
importo_complessivo_gara    4134
CLASSE_IMPORTO                 3
NOME_TEMATICA                 59
CODICE_MISSIONE                5
MISSIONE                       5
CODICE_COMPONENTE             13
COMPONENTE                    13
cod_mis_premiale              12
misura_premiale               12
CATEGORIA                      4
flag_quote                     3
quota_femminile               43
quota_giovanile               33
cod_mot_deroga                 9
mot_deroga                     9
data_pubblicazione           281
anno_pubblicazione             1
FLAG_URGENZA                   2
MOTIVO_URGENZA                 4
ESITO                          6
quota_femminile_disc           7
dtype: int64


,CIG,CUP,REGIONE,Regione,provincia,PROVINCIA,COMUNE,importo_complessivo_gara,CLASSE_IMPORTO,NOME_TEMATICA,...,quota_femminile,quota_giovanile,cod_mot_deroga,mot_deroga,data_pubblicazione,anno_pubblicazione,FLAG_URGENZA,MOTIVO_URGENZA,ESITO,quota_femminile_disc
85,A007ACE846,J84J21000040006,PIEMONTE,Piemonte,CUNEO,CUNEO,SAN MICHELE MONDOVI',33152.80,BASSA,M2C4: Tutela del territorio e della risorsa id...,...,15.0,30.0,2.0,MERCATO DI RIFERIMENTO,28/08/2023,2023,0,NaN,AGGIUDICATA,10.-15%
109,A000F0CB95,E28I22000160006,PUGLIA,Puglia,TARANTO,TARANTO,MASSAFRA,1809338.80,ALTA,"M6C1: Reti di prossimità, strutture e telemedi...",...,15.0,30.0,9.0,ADESIONE AD AQ/CONVENZIONE PRECEDENTE ALLE LIN...,02/08/2023,2023,0,NaN,NaN,10.-15%
110,A000F0CB95,E28I22000160006,PUGLIA,Puglia,TARANTO,TARANTO,MASSAFRA,1809338.80,ALTA,"M6C1: Reti di prossimità, strutture e telemedi...",...,15.0,30.0,9.0,ADESIONE AD AQ/CONVENZIONE PRECEDENTE ALLE LIN...,02/08/2023,2023,0,NaN,NaN,10.-15%
196,A003E86962,F86F22000000006,PUGLIA,Puglia,BARLETTA-ANDRIA-TRANI,BARLETTA-ANDRIA-TRANI,TRINITAPOLI,573100.00,MEDIA,M4C1: Potenziamento dell¿offerta dei servizi d...,...,15.0,30.0,5.0,SCARSA OCCUPAZIONE FEMMINILE NEL SETTORE,09/08/2023,2023,0,NaN,AGGIUDICATA,10.-15%
198,A00521B240,F14J22000670006,VENETO,Veneto,VENEZIA,VENEZIA,SANTO STINO DI LIVENZA,74450.00,BASSA,M2C4: Tutela del territorio e della risorsa id...,...,10.0,10.0,2.0,MERCATO DI RIFERIMENTO,18/08/2023,2023,0,NaN,AGGIUDICATA,5.-10%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273545,97633299F6,F71B21006060002,LOMBARDIA,Lombardia,LECCO,LECCO,MERATE,4589549.60,ALTA,"M6C1: Reti di prossimità, strutture e telemedi...",...,15.0,30.0,5.0,SCARSA OCCUPAZIONE FEMMINILE NEL SETTORE,04/07/2023,2023,0,NaN,NaN,10.-15%
273546,97633299F6,F71B21007000002,LOMBARDIA,Lombardia,LECCO,LECCO,MERATE,4589549.60,ALTA,"M6C1: Reti di prossimità, strutture e telemedi...",...,15.0,30.0,5.0,SCARSA OCCUPAZIONE FEMMINILE NEL SETTORE,04/07/2023,2023,0,NaN,NaN,10.-15%
273547,97633299F6,F71B21006060002,LOMBARDIA,Lombardia,LECCO,LECCO,MERATE,4589549.60,ALTA,"M6C1: Reti di prossimità, strutture e telemedi...",...,15.0,30.0,5.0,SCARSA OCCUPAZIONE FEMMINILE NEL SETTORE,04/07/2023,2023,0,NaN,NaN,10.-15%
273550,9941391760,D13D21000890005,MARCHE,Marche,ANCONA,ANCONA,FALCONARA MARITTIMA,4932873.81,ALTA,"M5C2: Infrastrutture sociali, famiglie, comuni...",...,15.0,30.0,5.0,SCARSA OCCUPAZIONE FEMMINILE NEL SETTORE,01/07/2023,2023,0,NaN,AGGIUDICATA,10.-15%


In [ ]:
p_basedati['PROVINCIA'].unique()
# where PROVINCIA=="Trento" | PROVINCIA=="Bolzano", p_basedati['REGIONE']=p_basedati['PROVINCIA']
p_basedati.loc[p_basedati.PROVINCIA == 'TRENTO', 'REGIONE'] = "TRENTO"
p_basedati.loc[p_basedati.PROVINCIA == 'BOLZANO', 'REGIONE'] = "BOLZANO"

In [17]:
p_basedati.loc[p_basedati.provincia == 'TRENTO', 'REGIONE'] = "TRENTO"
p_basedati.loc[p_basedati.provincia == 'BOLZANO', 'REGIONE'] = "BOLZANO"
p_basedati[p_basedati['REGIONE']=='TRENTINO-ALTO ADIGE'] # errori


,CIG,CUP,REGIONE,Regione,provincia,PROVINCIA,COMUNE,importo_complessivo_gara,CLASSE_IMPORTO,NOME_TEMATICA,...,quota_femminile,quota_giovanile,cod_mot_deroga,mot_deroga,data_pubblicazione,anno_pubblicazione,FLAG_URGENZA,MOTIVO_URGENZA,ESITO,quota_femminile_disc
4657,A002E3E9D7,J43C22000320006,TRENTINO-ALTO ADIGE,Friuli-Venezia Giulia,TRIESTE,NaN,NaN,1100.00,BASSA,M4C2: Dalla ricerca all'impresa - I1.5:Creazio...,...,NaN,NaN,8.0,ALTRO,07/08/2023,2023,0,NaN,NaN,NaN
6629,A0005B025A,J24E21001480001,TRENTINO-ALTO ADIGE,Lombardia,VARESE,NaN,NaN,28488436.76,ALTA,M3C1: Investimenti sulla rete ferroviaria - I1...,...,NaN,NaN,8.0,ALTRO,04/08/2023,2023,0,NaN,AGGIUDICATA,NaN
14480,A003406F2B,J43C22000320006,TRENTINO-ALTO ADIGE,Friuli-Venezia Giulia,TRIESTE,NaN,NaN,5600.00,BASSA,M4C2: Dalla ricerca all'impresa - I1.5:Creazio...,...,NaN,NaN,8.0,ALTRO,28/08/2023,2023,0,NaN,NaN,NaN
18173,A0118F28BA,J24E21001480001,TRENTINO-ALTO ADIGE,Piemonte,NOVARA,NaN,NaN,6115846.04,ALTA,M3C1: Investimenti sulla rete ferroviaria - I1...,...,NaN,NaN,8.0,ALTRO,26/09/2023,2023,0,NaN,NaN,NaN
18751,A010909795,J43C22000320006,TRENTINO-ALTO ADIGE,Friuli-Venezia Giulia,TRIESTE,NaN,NaN,32000.00,BASSA,M4C2: Dalla ricerca all'impresa - I1.5:Creazio...,...,NaN,NaN,1.0,IMPORTO RIDOTTO DEL CONTRATTO,21/09/2023,2023,0,NaN,AGGIUDICATA,NaN
21262,9963638E2C,F43C22000200006,TRENTINO-ALTO ADIGE,NaN,NaN,NaN,NaN,2675.00,BASSA,M4C2: Dalla ricerca all'impresa - I1.5:Creazio...,...,NaN,NaN,1.0,IMPORTO RIDOTTO DEL CONTRATTO,27/09/2023,2023,0,NaN,NaN,NaN
24197,A005738A76,J43C22000320006,TRENTINO-ALTO ADIGE,Friuli-Venezia Giulia,TRIESTE,NaN,NaN,32786.88,BASSA,M4C2: Dalla ricerca all'impresa - I1.5:Creazio...,...,NaN,NaN,8.0,ALTRO,21/09/2023,2023,0,NaN,NaN,NaN
24513,A010D7B2B2,J34E21001820001,TRENTINO-ALTO ADIGE,NaN,NaN,NaN,NaN,7052537.68,ALTA,M3C1: Investimenti sulla rete ferroviaria - I1...,...,NaN,NaN,8.0,ALTRO,20/09/2023,2023,0,NaN,NaN,NaN
29363,A00FDD372F,F43C22000200006,TRENTINO-ALTO ADIGE,Veneto,VENEZIA,NaN,NaN,40000.00,BASSA,M4C2: Dalla ricerca all'impresa - I1.5:Creazio...,...,NaN,NaN,1.0,IMPORTO RIDOTTO DEL CONTRATTO,15/09/2023,2023,0,NaN,NaN,NaN
32274,A0121208C0,J24E21001480001,TRENTINO-ALTO ADIGE,Lombardia,VARESE,NaN,NaN,6626396.70,ALTA,M3C1: Investimenti sulla rete ferroviaria - I1...,...,NaN,NaN,8.0,ALTRO,26/09/2023,2023,0,NaN,NaN,NaN


In [20]:
p_basedati['REGIONE'].unique()

array(['LIGURIA', 'CAMPANIA', nan, 'ABRUZZO', 'CALABRIA', 'LAZIO',
       'LOMBARDIA', 'PIEMONTE', 'UMBRIA', 'TOSCANA', 'EMILIA-ROMAGNA',
       'FRIULI-VENEZIA GIULIA', 'MARCHE', 'SARDEGNA', 'TRENTO', 'VENETO',
       'BOLZANO', 'BASILICATA', 'PUGLIA', 'SICILIA', 'MOLISE',
       "VALLE D'AOSTA", 'TRENTINO-ALTO ADIGE'], dtype=object)

In [19]:
p_basedati.to_csv("basedati.csv", index = False, sep=';')

In [ ]:
# NON SERVE
# pre-processing p04
# re-encode quote
# preprocessing encoding righe
p04['quota_femminile'] = p04['quota_femminile'].astype("Int32")
p04['quota_giovanile'] = p04['quota_giovanile'].astype("Int32")
p04['quota_femminile'] = p04['quota_femminile'].apply(lambda x: x / 10 ** (len((str(x))) - 2))
p04['quota_giovanile'] = p04['quota_giovanile'].apply(lambda x: x / 10 ** (len((str(x))) - 2))
p04['quota_femminile'] = p04['quota_femminile'].astype("Float64")
p04['quota_giovanile'] = p04['quota_giovanile'].astype("Float64")

regioni - taa

In [27]:
p_basedati = pd.read_csv("basedati.csv",delimiter=';')

In [22]:
p_basedati[(p_basedati['REGIONE']!=p_basedati['Regione'].str.upper())&(p_basedati['Regione'].notna())]['CIG'].nunique() # case sentitive

24882

In [28]:
#p_basedati[p_basedati['REGIONE']=='TRENTINO-ALTO ADIGE'].REGIONE = p_basedati[p_basedati['REGIONE']=='TRENTINO-ALTO ADIGE'].Regione
#p_basedati.loc[p_basedati.REGIONE == 'TRENTINO-ALTO ADIGE', 'REGIONE'] = p_basedati.loc[p_basedati.REGIONE == 'TRENTINO-ALTO ADIGE', 'Regione'.upper()] 
p_basedati['Regione'] = p_basedati['Regione'].str.upper()
p_basedati.loc[p_basedati.REGIONE == 'TRENTINO-ALTO ADIGE', 'REGIONE'] = p_basedati.loc[p_basedati.REGIONE == 'TRENTINO-ALTO ADIGE', 'Regione'] 


In [31]:
p_basedati[p_basedati['REGIONE']=='TRENTINO-ALTO ADIGE']
p_basedati['REGIONE'].unique()

array(['LIGURIA', 'CAMPANIA', nan, 'ABRUZZO', 'CALABRIA', 'LAZIO',
       'LOMBARDIA', 'PIEMONTE', 'UMBRIA', 'TOSCANA', 'EMILIA-ROMAGNA',
       'FRIULI-VENEZIA GIULIA', 'MARCHE', 'SARDEGNA', 'TRENTO', 'VENETO',
       'BOLZANO', 'BASILICATA', 'PUGLIA', 'SICILIA', 'MOLISE',
       "VALLE D'AOSTA"], dtype=object)

In [32]:
p_basedati

,CIG,CUP,REGIONE,Regione,provincia,PROVINCIA,COMUNE,importo_complessivo_gara,CLASSE_IMPORTO,NOME_TEMATICA,...,quota_femminile,quota_giovanile,cod_mot_deroga,mot_deroga,data_pubblicazione,anno_pubblicazione,FLAG_URGENZA,MOTIVO_URGENZA,ESITO,quota_femminile_disc
0,A0011137E1,G79J19001530007,LIGURIA,NaN,NaN,IMPERIA,SAN REMO,21131.90,BASSA,M2C4: Tutela del territorio e della risorsa id...,...,NaN,NaN,1.0,IMPORTO RIDOTTO DEL CONTRATTO,02/08/2023,2023,0,NaN,AGGIUDICATA,NaN
1,A0011137E1,G79J19001530007,LIGURIA,NaN,NaN,IMPERIA,IMPERIA,21131.90,BASSA,M2C4: Tutela del territorio e della risorsa id...,...,NaN,NaN,1.0,IMPORTO RIDOTTO DEL CONTRATTO,02/08/2023,2023,0,NaN,AGGIUDICATA,NaN
2,A0011137E1,G79J19001530007,LIGURIA,NaN,NaN,IMPERIA,SAN BARTOLOMEO AL MARE,21131.90,BASSA,M2C4: Tutela del territorio e della risorsa id...,...,NaN,NaN,1.0,IMPORTO RIDOTTO DEL CONTRATTO,02/08/2023,2023,0,NaN,AGGIUDICATA,NaN
3,A0011137E1,G79J19001530007,LIGURIA,NaN,NaN,IMPERIA,CERVO,21131.90,BASSA,M2C4: Tutela del territorio e della risorsa id...,...,NaN,NaN,1.0,IMPORTO RIDOTTO DEL CONTRATTO,02/08/2023,2023,0,NaN,AGGIUDICATA,NaN
4,A0011137E1,G79J19001530007,LIGURIA,NaN,NaN,SAVONA,ANDORA,21131.90,BASSA,M2C4: Tutela del territorio e della risorsa id...,...,NaN,NaN,1.0,IMPORTO RIDOTTO DEL CONTRATTO,02/08/2023,2023,0,NaN,AGGIUDICATA,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273874,A001EFE42A,J81D22000100002,LOMBARDIA,LOMBARDIA,BERGAMO,BERGAMO,OSIO SOTTO,149800.00,MEDIA,M2C4: Tutela del territorio e della risorsa id...,...,NaN,NaN,NaN,NaN,03/08/2023,2023,0,NaN,NaN,NaN
273875,A000A6FCFD,J67H21003620005,EMILIA-ROMAGNA,EMILIA-ROMAGNA,FERRARA,FERRARA,COPPARO,1262.42,BASSA,"M5C2: Infrastrutture sociali, famiglie, comuni...",...,NaN,NaN,8.0,ALTRO,01/08/2023,2023,0,NaN,NaN,NaN
273876,99600068F5,C44D22003310006,SICILIA,SICILIA,CATANIA,CATANIA,SCORDIA,101732.97,MEDIA,M4C1: Potenziamento dell¿offerta dei servizi d...,...,NaN,NaN,8.0,ALTRO,14/08/2023,2023,1,PROCEDURA NEGOZIATA PER ESTREMA URGENZA,NaN,NaN
273877,9946229FCF,G27H22000720001,NaN,ABRUZZO,TERAMO,NaN,NaN,668715.94,MEDIA,NaN,...,NaN,NaN,1.0,IMPORTO RIDOTTO DEL CONTRATTO,13/07/2023,2023,0,NaN,AGGIUDICATA,NaN


In [33]:
p_basedati.to_csv("basedati.csv", index = False, sep=';')

In [2]:
import pandas as pd
import numpy as np

In [3]:
p_basedati = pd.read_csv("basedati.csv",delimiter=';')

In [4]:
p_basedati

,CIG,CUP,REGIONE,Regione,provincia,PROVINCIA,COMUNE,importo_complessivo_gara,CLASSE_IMPORTO,NOME_TEMATICA,...,quota_femminile,quota_giovanile,cod_mot_deroga,mot_deroga,data_pubblicazione,anno_pubblicazione,FLAG_URGENZA,MOTIVO_URGENZA,ESITO,quota_femminile_disc
0,A0011137E1,G79J19001530007,LIGURIA,NaN,NaN,IMPERIA,SAN REMO,21131.90,BASSA,M2C4: Tutela del territorio e della risorsa id...,...,NaN,NaN,1.0,IMPORTO RIDOTTO DEL CONTRATTO,02/08/2023,2023,0,NaN,AGGIUDICATA,NaN
1,A0011137E1,G79J19001530007,LIGURIA,NaN,NaN,IMPERIA,IMPERIA,21131.90,BASSA,M2C4: Tutela del territorio e della risorsa id...,...,NaN,NaN,1.0,IMPORTO RIDOTTO DEL CONTRATTO,02/08/2023,2023,0,NaN,AGGIUDICATA,NaN
2,A0011137E1,G79J19001530007,LIGURIA,NaN,NaN,IMPERIA,SAN BARTOLOMEO AL MARE,21131.90,BASSA,M2C4: Tutela del territorio e della risorsa id...,...,NaN,NaN,1.0,IMPORTO RIDOTTO DEL CONTRATTO,02/08/2023,2023,0,NaN,AGGIUDICATA,NaN
3,A0011137E1,G79J19001530007,LIGURIA,NaN,NaN,IMPERIA,CERVO,21131.90,BASSA,M2C4: Tutela del territorio e della risorsa id...,...,NaN,NaN,1.0,IMPORTO RIDOTTO DEL CONTRATTO,02/08/2023,2023,0,NaN,AGGIUDICATA,NaN
4,A0011137E1,G79J19001530007,LIGURIA,NaN,NaN,SAVONA,ANDORA,21131.90,BASSA,M2C4: Tutela del territorio e della risorsa id...,...,NaN,NaN,1.0,IMPORTO RIDOTTO DEL CONTRATTO,02/08/2023,2023,0,NaN,AGGIUDICATA,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273874,A001EFE42A,J81D22000100002,LOMBARDIA,LOMBARDIA,BERGAMO,BERGAMO,OSIO SOTTO,149800.00,MEDIA,M2C4: Tutela del territorio e della risorsa id...,...,NaN,NaN,NaN,NaN,03/08/2023,2023,0,NaN,NaN,NaN
273875,A000A6FCFD,J67H21003620005,EMILIA-ROMAGNA,EMILIA-ROMAGNA,FERRARA,FERRARA,COPPARO,1262.42,BASSA,"M5C2: Infrastrutture sociali, famiglie, comuni...",...,NaN,NaN,8.0,ALTRO,01/08/2023,2023,0,NaN,NaN,NaN
273876,99600068F5,C44D22003310006,SICILIA,SICILIA,CATANIA,CATANIA,SCORDIA,101732.97,MEDIA,M4C1: Potenziamento dell¿offerta dei servizi d...,...,NaN,NaN,8.0,ALTRO,14/08/2023,2023,1,PROCEDURA NEGOZIATA PER ESTREMA URGENZA,NaN,NaN
273877,9946229FCF,G27H22000720001,NaN,ABRUZZO,TERAMO,NaN,NaN,668715.94,MEDIA,NaN,...,NaN,NaN,1.0,IMPORTO RIDOTTO DEL CONTRATTO,13/07/2023,2023,0,NaN,AGGIUDICATA,NaN


prima

In [99]:
# prima
# con tutti i campi p02, con tutti i campi P05
p07 = leftnew_3412_complete
# con tutti i campi p02, i campi di P05 sono già selezionati
p07_rid = leftnew_3412
# selezione di campi
p07_base = p07_rid[['CIG', 'importo_complessivo_gara','CLASSE_IMPORTO',
       'cod_mis_premiale', 'misura_premiale', 'CATEGORIA', 'flag_quote',
       'quota_femminile', 'quota_giovanile', 'cod_mot_deroga', 'mot_deroga',
       'CUP', 'REGIONE', 'PROVINCIA', 'COMUNE','ANNO_DECISIONE','data_pubblicazione', 'COSTO_PROGETTO', 'IMPORTO_FINANZIAMENTO', 'CODICE_MISSIONE', 'COMPONENTE',
       'INVESTIMENTO', 'MISSIONE']]

In [105]:
print(p07_base.shape)
print(p07_rid.shape)
print(p07.shape)

(273879, 23)
(273879, 56)
(273879, 113)


In [101]:
# è un problema di match coi comuni?
p07_rid[p07_rid['CIG']=="A0011137E1"]

,CIG,importo_complessivo_gara,CLASSE_IMPORTO,luogo_istat,provincia,Denominazione sovracomunale,Regione,data_pubblicazione,cod_mis_premiale,misura_premiale,...,DATA_GENERAZIONE_COMPLETO,COSTO_PROGETTO,IMPORTO_FINANZIAMENTO,STATO_PROGETTO,TIPO_PROGETTO,CODICE_COMPONENTE,CODICE_MISSIONE,COMPONENTE,INVESTIMENTO,MISSIONE
0,A0011137E1,21131.90,BASSA,NaN,NaN,NaN,NaN,2023-08-02,NaN,NaN,...,2021-11-29,29500000.00,29500000.00,Attivo,Progetto Normale,M2C4,M2,Tutela del territorio e della risorsa idrica,I4.1:Investimenti in infrastrutture idriche pr...,RIVOLUZIONE VERDE E TRANSIZIONE ECOLOGICA
1,A0011137E1,21131.90,BASSA,NaN,NaN,NaN,NaN,2023-08-02,NaN,NaN,...,2021-11-29,29500000.00,29500000.00,Attivo,Progetto Normale,M2C4,M2,Tutela del territorio e della risorsa idrica,I4.1:Investimenti in infrastrutture idriche pr...,RIVOLUZIONE VERDE E TRANSIZIONE ECOLOGICA
2,A0011137E1,21131.90,BASSA,NaN,NaN,NaN,NaN,2023-08-02,NaN,NaN,...,2021-11-29,29500000.00,29500000.00,Attivo,Progetto Normale,M2C4,M2,Tutela del territorio e della risorsa idrica,I4.1:Investimenti in infrastrutture idriche pr...,RIVOLUZIONE VERDE E TRANSIZIONE ECOLOGICA
3,A0011137E1,21131.90,BASSA,NaN,NaN,NaN,NaN,2023-08-02,NaN,NaN,...,2021-11-29,29500000.00,29500000.00,Attivo,Progetto Normale,M2C4,M2,Tutela del territorio e della risorsa idrica,I4.1:Investimenti in infrastrutture idriche pr...,RIVOLUZIONE VERDE E TRANSIZIONE ECOLOGICA
4,A0011137E1,21131.90,BASSA,NaN,NaN,NaN,NaN,2023-08-02,NaN,NaN,...,2021-11-29,29500000.00,29500000.00,Attivo,Progetto Normale,M2C4,M2,Tutela del territorio e della risorsa idrica,I4.1:Investimenti in infrastrutture idriche pr...,RIVOLUZIONE VERDE E TRANSIZIONE ECOLOGICA
5,A0011137E1,21131.90,BASSA,NaN,NaN,NaN,NaN,2023-08-02,NaN,NaN,...,2021-11-29,29500000.00,29500000.00,Attivo,Progetto Normale,M2C4,M2,Tutela del territorio e della risorsa idrica,I4.1:Investimenti in infrastrutture idriche pr...,RIVOLUZIONE VERDE E TRANSIZIONE ECOLOGICA


In [103]:
p07_base.to_csv("tmp_finale_base.csv", index = False, sep=';')
p07_rid.to_csv("tmp_finale_ridotto.csv", index = False, sep=';')
p07.to_csv("tmp_finale.csv", index = False, sep=';')